In [1]:
import tensorflow as tf

import numpy as np
import os
import time

In [2]:
path_to_file = "/Users/AnaPSilva/Documents/Ana/Ironhack/Bootcamp/Final_Project/Data/Rap/eminem.txt"

In [3]:
# Read, then decode for py2 compat.
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print(f'Length of text: {len(text)} characters')

Length of text: 926405 characters


In [4]:
# Take a look at the first 250 characters in text
print(text[:250])

Look, I was gonna go easy on you not to hurt your feelings
But I'm only going to get this one chance
(Six minutes, six minutes)
Something's wrong, I can feel it
(Six minutes, six minutes, Slim Shady, you're on)
Just a feeling I've got
Like something'


In [5]:
# The unique characters in the file
vocab = sorted(set(text))
print(f'{len(vocab)} unique characters')

110 unique characters


In [6]:
## Before training, you need to convert the strings to a numerical representation.
## convert each character into a numeric ID. 
## It just needs the text to be split into tokens first.
example_texts = ['abcdefg', 'xyz']

chars = tf.strings.unicode_split(example_texts, input_encoding='UTF-8')
chars

2022-03-14 03:01:19.243618: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

In [7]:
## from tokens to character IDs
ids_from_chars = tf.keras.layers.StringLookup(
    vocabulary=list(vocab), mask_token=None)
ids = ids_from_chars(chars)
ids

<tf.RaggedTensor [[60, 61, 62, 63, 64, 65, 66], [83, 84, 85]]>

In [8]:
## it will also be important to invert this representation and 
## recover human-readable strings from it.
chars_from_ids = tf.keras.layers.StringLookup(
    vocabulary=ids_from_chars.get_vocabulary(), invert=True, mask_token=None)

In [9]:
chars = chars_from_ids(ids)
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

In [10]:
## join the characters back into strings.
tf.strings.reduce_join(chars, axis=-1).numpy()

array([b'abcdefg', b'xyz'], dtype=object)

In [11]:
def text_from_ids(ids):
  """join the characters back into strings"""
  return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)

#### Create training examples and targets

In [12]:
## convert the text vector into a stream of character indices.
all_ids = ids_from_chars(tf.strings.unicode_split(text, 'UTF-8'))
all_ids

<tf.Tensor: shape=(926405,), dtype=int64, numpy=array([42, 74, 74, ..., 60, 72, 64])>

In [13]:
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)
for ids in ids_dataset.take(10):
    print(chars_from_ids(ids).numpy().decode('utf-8'))

L
o
o
k
,
 
I
 
w
a


In [14]:
seq_length = 100
examples_per_epoch = len(text)//(seq_length+1)
examples_per_epoch

9172

In [15]:
## The batch method lets you easily convert 
## these individual characters to sequences of the desired size.
sequences = ids_dataset.batch(seq_length+1, drop_remainder=True)

for seq in sequences.take(1):
  print(chars_from_ids(seq))

tf.Tensor(
[b'L' b'o' b'o' b'k' b',' b' ' b'I' b' ' b'w' b'a' b's' b' ' b'g' b'o'
 b'n' b'n' b'a' b' ' b'g' b'o' b' ' b'e' b'a' b's' b'y' b' ' b'o' b'n'
 b' ' b'y' b'o' b'u' b' ' b'n' b'o' b't' b' ' b't' b'o' b' ' b'h' b'u'
 b'r' b't' b' ' b'y' b'o' b'u' b'r' b' ' b'f' b'e' b'e' b'l' b'i' b'n'
 b'g' b's' b'\n' b'B' b'u' b't' b' ' b'I' b"'" b'm' b' ' b'o' b'n' b'l'
 b'y' b' ' b'g' b'o' b'i' b'n' b'g' b' ' b't' b'o' b' ' b'g' b'e' b't'
 b' ' b't' b'h' b'i' b's' b' ' b'o' b'n' b'e' b' ' b'c' b'h' b'a' b'n'
 b'c' b'e' b'\n'], shape=(101,), dtype=string)


In [16]:
## It's easier to see what this is doing if you join the 
## tokens back into strings

for seq in sequences.take(5):
  print(text_from_ids(seq).numpy())

b"Look, I was gonna go easy on you not to hurt your feelings\nBut I'm only going to get this one chance\n"
b"(Six minutes, six minutes)\nSomething's wrong, I can feel it\n(Six minutes, six minutes, Slim Shady, yo"
b"u're on)\nJust a feeling I've got\nLike something's about to happen\nBut I don't know what\nIf that means"
b", what I think it means, we're in trouble\nBig trouble. And if he is as bananas as you say\nI'm not tak"
b"ing any chances\nYou are just what the doc ordered\n\nI'm beginning to feel like a Rap God, Rap God\nAll "


In [17]:
def split_input_target(sequence):
    """takes a sequence as input, duplicates, 
    and shifts it to align the input and label for each timestep"""
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text

In [18]:
dataset = sequences.map(split_input_target)
for input_example, target_example in dataset.take(1):
    print("Input :", text_from_ids(input_example).numpy())
    print("Target:", text_from_ids(target_example).numpy())

Input : b"Look, I was gonna go easy on you not to hurt your feelings\nBut I'm only going to get this one chance"
Target: b"ook, I was gonna go easy on you not to hurt your feelings\nBut I'm only going to get this one chance\n"


#### Create training batches

In [19]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))

dataset

<PrefetchDataset shapes: ((64, 100), (64, 100)), types: (tf.int64, tf.int64)>

#### Build The Model

- **tf.keras.layers.Embedding:** The input layer. A trainable lookup table that will map each character-ID to a vector with embedding_dim dimensions;
- **tf.keras.layers.GRU:** A type of RNN with size units=rnn_units (You can also use an LSTM layer here.)
- **tf.keras.layers.Dense:** The output layer, with vocab_size outputs. It outputs one logit for each character in the vocabulary. These are the log-likelihood of each character according to the model.

In [20]:
# Length of the vocabulary in chars
vocab_size = len(vocab)
display(vocab_size)
# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

110

In [21]:
class MyModel(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, rnn_units):
    super().__init__(self)
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(rnn_units,
                                   return_sequences=True,
                                   return_state=True)
    self.dense = tf.keras.layers.Dense(vocab_size)

  def call(self, inputs, states=None, return_state=False, training=False):
    x = inputs
    x = self.embedding(x, training=training)
    if states is None:
      states = self.gru.get_initial_state(x)
    x, states = self.gru(x, initial_state=states, training=training)
    x = self.dense(x, training=training)

    if return_state:
      return x, states
    else:
      return x

In [22]:
model = MyModel(
    # Making sure the vocabulary size matches the `StringLookup` layers.
    vocab_size=len(ids_from_chars.get_vocabulary()),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

#### Try the model

In [23]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 111) # (batch_size, sequence_length, vocab_size)


In [24]:
model.summary()

Model: "my_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       multiple                  28416     
                                                                 
 gru (GRU)                   multiple                  3938304   
                                                                 
 dense (Dense)               multiple                  113775    
                                                                 
Total params: 4,080,495
Trainable params: 4,080,495
Non-trainable params: 0
_________________________________________________________________


In [25]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices, axis=-1).numpy()

## This gives us, at each timestep, a prediction of the next character index
sampled_indices

array([ 40,  89,  91,  42,  93,  17,  76,  89,  23, 103,  68,   9,  84,
       104,  44,  71,  51,  83,  54, 107,  30,  99,   7,  72,  38,  11,
        17,  82,  17,  98,   8, 109,  96,   8,  67,  68, 100,  65,  44,
        79,  24,  60,  79,  47,  90,  93,  43,   1,  17,  60,  91,  22,
         4,   2,  17,  20,  98,  78,  94,  16,  94,  93,  20,   4,  98,
         6,  60,  53,  50,  80,  17, 101,  42,  15,  99,  56,  36,  44,
         6,  10,  88,  31,  52,   3,  29,   3,  90,  76,  62,  82,  74,
        38, 101,  86,   4,  55,  88, 108,   1,  93])

In [26]:
## Decode these to see the text predicted by this untrained model
print("Input:\n", text_from_ids(input_example_batch[0]).numpy())
print()
print("Next Char Predictions:\n", text_from_ids(sampled_indices).numpy())

Input:
 b'\nA big Chinese nigga, screamin "Kuniva yo yo.."\nI leave ya face leakin, run up in church \nand smack '

Next Char Predictions:
 b"J\xc3\x85\xc3\xa0L\xc3\xa2/q\xc3\x855\xe2\x80\x93i'y\xe2\x80\x94NlUxX\xe2\x80\x9c?\xc3\xb6%mH)/w/\xc3\xb3&\xe2\x80\xa6\xc3\xa9&hi\xc3\xbafNt6atQ\xc3\x9f\xc3\xa2M\n/a\xc3\xa04!\r/2\xc3\xb3s\xc3\xa4.\xc3\xa4\xc3\xa22!\xc3\xb3$aWTu/\xc3\xbcL-\xc3\xb6ZFN$(\xc2\xa9AV ; \xc3\x9fqcwoH\xc3\xbc{!Y\xc2\xa9\xe2\x80\x9d\n\xc3\xa2"


#### Train the model
- At this point the problem can be treated as a standard classification problem. Given the previous RNN state, and the input this time step, predict the class of the next character.

In [27]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

In [28]:
example_batch_mean_loss = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("Mean loss:        ", example_batch_mean_loss)

Prediction shape:  (64, 100, 111)  # (batch_size, sequence_length, vocab_size)
Mean loss:         tf.Tensor(4.709812, shape=(), dtype=float32)


In [29]:
tf.exp(example_batch_mean_loss).numpy()

111.0313

In [30]:
## training procedure using the tf.keras.Model.compile method. 
## Use tf.keras.optimizers.Adam with default arguments and the loss function.
model.compile(optimizer='adam', loss=loss)

In [31]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_rap_eminem_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [32]:
EPOCHS = 35
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/35
143/143 [==============================] - 473s 3s/step - loss: 2.9159
Epoch 2/35
143/143 [==============================] - 434s 3s/step - loss: 2.1703
Epoch 3/35
143/143 [==============================] - 431s 3s/step - loss: 1.9063
Epoch 4/35
143/143 [==============================] - 434s 3s/step - loss: 1.7402
Epoch 5/35
143/143 [==============================] - 434s 3s/step - loss: 1.6219
Epoch 6/35
143/143 [==============================] - 434s 3s/step - loss: 1.5311
Epoch 7/35
143/143 [==============================] - 434s 3s/step - loss: 1.4554
Epoch 8/35
143/143 [==============================] - 434s 3s/step - loss: 1.3878
Epoch 9/35
143/143 [==============================] - 435s 3s/step - loss: 1.3238
Epoch 10/35
143/143 [==============================] - 437s 3s/step - loss: 1.2600
Epoch 11/35
143/143 [==============================] - 437s 3s/step - loss: 1.1944
Epoch 12/35
143/143 [==============================] - 443s 3s/step - loss: 1.1288
Epoch 13/35
1

In [37]:
class OneStep(tf.keras.Model):
  def __init__(self, model, chars_from_ids, ids_from_chars, temperature=1.0):
    super().__init__()
    self.temperature = temperature
    self.model = model
    self.chars_from_ids = chars_from_ids
    self.ids_from_chars = ids_from_chars

    # Create a mask to prevent "[UNK]" from being generated.
    skip_ids = self.ids_from_chars(['[UNK]'])[:, None]
    sparse_mask = tf.SparseTensor(
        # Put a -inf at each bad index.
        values=[-float('inf')]*len(skip_ids),
        indices=skip_ids,
        # Match the shape to the vocabulary
        dense_shape=[len(ids_from_chars.get_vocabulary())])
    self.prediction_mask = tf.sparse.to_dense(sparse_mask)

  @tf.function
  def generate_one_step(self, inputs, states=None):
    # Convert strings to token IDs.
    input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
    input_ids = self.ids_from_chars(input_chars).to_tensor()

    # Run the model.
    # predicted_logits.shape is [batch, char, next_char_logits]
    predicted_logits, states = self.model(inputs=input_ids, states=states,
                                          return_state=True)
    # Only use the last prediction.
    predicted_logits = predicted_logits[:, -1, :]
    predicted_logits = predicted_logits/self.temperature
    # Apply the prediction mask: prevent "[UNK]" from being generated.
    predicted_logits = predicted_logits + self.prediction_mask

    # Sample the output logits to generate token IDs.
    predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
    predicted_ids = tf.squeeze(predicted_ids, axis=-1)

    # Convert from token ids to characters
    predicted_chars = self.chars_from_ids(predicted_ids)

    # Return the characters and model state.
    return predicted_chars, states

In [38]:
one_step_model = OneStep(model, chars_from_ids, ids_from_chars)

In [39]:
start = time.time()
states = None
next_char = tf.constant(['a'])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)

am just like her
Everything I do wan disfed the world off

The way you move it, you know what can I do?
Mo passionates at a bride colired
So I won't be sator, sew, but it's stolen
Tryna pulk packetta what my exact turks and saw
But I've never seen down the wind the clock
I just got my onf my weenie is much before I know that you should
Long as you know you're unsour
No more piece of the indumonaly
Million dollars ain't it hits around and do attention
I'm just watching to conscience
I'm turning back to the road, even though me and Doc hair hed man
If at all the flame guess white ballin'
They're hanger intertroned
That though for murdering has married to explain
I ain't murdered nobody
I stand there and wait a motherfuckin' table
We're only life it
Got another lawndom
But if you better afraid on the door
Somebody's life, like a swear-to-carribreat, 
I'm Slim Shady, a thousand beat
"But she said "No, I'm here
And I ain't just playin and nails your cheek? Me?
My eye 'em down, down (Wanowa 

In [40]:
tf.saved_model.save(one_step_model, 'Eminem')

INFO:tensorflow:Assets written to: Eminem/assets


INFO:tensorflow:Assets written to: Eminem/assets
